In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error



# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# reading each csv file as data frame
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# list 
dfs = [df_21, df_22, df_23]

# concat
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
df = df[~df['STTN_NM'].isin(['이동차', '부산항'])]
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 타입 확인 및 변환
df[numeric_columns] = df[numeric_columns].astype(float)
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Hour'] = df['Hour'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(df['MSR_DT'][X_train.index])
X_test.index = pd.to_datetime(df['MSR_DT'][X_test.index])
y_train.index = pd.to_datetime(df['MSR_DT'][y_train.index])
y_test.index = pd.to_datetime(df['MSR_DT'][y_test.index])

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델ß
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


KeyError: 'Year'

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error


# 열 이름 확인
print("Columns in the DataFrame:", df.columns)

# 불필요한 열 제거
df.drop(columns=['STTN_NM'], inplace=True, errors='ignore')  # 'ignore' 옵션으로 오류 발생 방지

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 타입 확인 및 변환
df[numeric_columns] = df[numeric_columns].astype(float)
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Hour'] = df['Hour'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(df['MSR_DT'][X_train.index])
X_test.index = pd.to_datetime(df['MSR_DT'][X_test.index])
y_train.index = pd.to_datetime(df['MSR_DT'][y_train.index])
y_test.index = pd.to_datetime(df['MSR_DT'][y_test.index])

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Columns in the DataFrame: Index(['MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2'], dtype='object')


KeyError: 'Year'

In [4]:
print(df.columns)

Index(['MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2'], dtype='object')


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error


# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# reading each csv file as data frame
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# list 
dfs = [df_21, df_22, df_23]

# concat
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
# df = df[~df['STTN_NM'].isin(['이동차', '부산항'])]
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

for col in df.columns:
    # if df[col].dtype != 'datetime64[ns]':
    df[col] = df[col].interpolate(method='linear', limit_direction='both')

# # MSR_DT를 datetime 형식으로 변환
# df.loc[:, 'MSR_DT'] = pd.to_datetime(df['MSR_DT'], format='%Y%m%d%H', errors='coerce')

# df.loc[:, 'Year'] = df['MSR_DT'].dt.year
# df.loc[:, 'Month'] = df['MSR_DT'].dt.month
# df.loc[:, 'Hour'] = df['MSR_DT'].dt.hour

df['MSR_DT'] = pd.to_datetime(df['MSR_DT'], format='%Y%m%d%H', errors='coerce')
df['Year'] = df['MSR_DT'].dt.year
df['Month'] = df['MSR_DT'].dt.month
df['Hour'] = df['MSR_DT'].dt.hour

# 열 이름 확인
print("Columns in the DataFrame:", df.columns)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 타입 확인 및 변환
df[numeric_columns] = df[numeric_columns].astype(float)
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Hour'] = df['Hour'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(df['MSR_DT'][X_train.index])
X_test.index = pd.to_datetime(df['MSR_DT'][X_test.index])
y_train.index = pd.to_datetime(df['MSR_DT'][y_train.index])
y_test.index = pd.to_datetime(df['MSR_DT'][y_test.index])

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Columns in the DataFrame: Index(['MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2', 'Year', 'Month',
       'Hour'],
      dtype='object')


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# 각 CSV 파일 읽기
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# 데이터프레임 리스트
dfs = [df_21, df_22, df_23]

# 데이터프레임 결합
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

# MSR_DT를 datetime 형식으로 변환
df['MSR_DT'] = pd.to_datetime(df['MSR_DT'], format='%Y%m%d%H', errors='coerce')

# 'Year', 'Month', 'Hour' 열 추가
df['Year'] = df['MSR_DT'].dt.year
df['Month'] = df['MSR_DT'].dt.month
df['Hour'] = df['MSR_DT'].dt.hour

# 결측값 확인 및 처리
# 모든 결측값을 중간값으로 대체
for col in df.columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# 열 이름 확인
print("Columns in the DataFrame:", df.columns)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 타입 확인 및 변환
df[numeric_columns] = df[numeric_columns].astype(float)
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Hour'] = df['Hour'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(df['MSR_DT'][X_train.index])
X_test.index = pd.to_datetime(df['MSR_DT'][X_test.index])
y_train.index = pd.to_datetime(df['MSR_DT'][y_train.index])
y_test.index = pd.to_datetime(df['MSR_DT'][y_test.index])

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Columns in the DataFrame: Index(['MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2', 'Year', 'Month',
       'Hour'],
      dtype='object')
Shape of X_train: (897875, 7)
Shape of X_test: (224469, 7)
Shape of y_train: (897875, 2)
Shape of y_test: (224469, 2)


/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-

ValueError: cannot reindex on an axis with duplicate labels

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# 각 CSV 파일 읽기
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# 데이터프레임 리스트
dfs = [df_21, df_22, df_23]

# 데이터프레임 결합
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

# MSR_DT를 datetime 형식으로 변환
df['MSR_DT'] = pd.to_datetime(df['MSR_DT'], format='%Y%m%d%H', errors='coerce')

# 'Year', 'Month', 'Hour' 열 추가
df['Year'] = df['MSR_DT'].dt.year
df['Month'] = df['MSR_DT'].dt.month
df['Hour'] = df['MSR_DT'].dt.hour

# 결측값 확인 및 처리
for col in df.columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# 열 이름 확인
print("Columns in the DataFrame:", df.columns)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 타입 확인 및 변환
df[numeric_columns] = df[numeric_columns].astype(float)
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Hour'] = df['Hour'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 중복된 인덱스 확인 및 제거
X_train = X_train[~X_train.index.duplicated(keep='first')]
X_test = X_test[~X_test.index.duplicated(keep='first')]
y_train = y_train[~y_train.index.duplicated(keep='first')]
y_test = y_test[~y_test.index.duplicated(keep='first')]

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(df['MSR_DT'][X_train.index])
X_test.index = pd.to_datetime(df['MSR_DT'][X_test.index])
y_train.index = pd.to_datetime(df['MSR_DT'][y_train.index])
y_test.index = pd.to_datetime(df['MSR_DT'][y_test.index])

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Columns in the DataFrame: Index(['MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2', 'Year', 'Month',
       'Hour'],
      dtype='object')
Shape of X_train: (897875, 7)
Shape of X_test: (224469, 7)
Shape of y_train: (897875, 2)
Shape of y_test: (224469, 2)


/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-

ValueError: cannot reindex on an axis with duplicate labels

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# 각 CSV 파일 읽기
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# 데이터프레임 리스트
dfs = [df_21, df_22, df_23]

# 데이터프레임 결합
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

# MSR_DT를 datetime 형식으로 변환
df['MSR_DT'] = pd.to_datetime(df['MSR_DT'], format='%Y%m%d%H', errors='coerce')

# 'Year', 'Month', 'Hour' 열 추가
df['Year'] = df['MSR_DT'].dt.year
df['Month'] = df['MSR_DT'].dt.month
df['Hour'] = df['MSR_DT'].dt.hour

# 결측값 확인 및 처리
for col in df.columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# 열 이름 확인
print("Columns in the DataFrame:", df.columns)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 타입 확인 및 변환
df[numeric_columns] = df[numeric_columns].astype(float)
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Hour'] = df['Hour'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(df.loc[X_train.index, 'MSR_DT'])
X_test.index = pd.to_datetime(df.loc[X_test.index, 'MSR_DT'])
y_train.index = pd.to_datetime(df.loc[y_train.index, 'MSR_DT'])
y_test.index = pd.to_datetime(df.loc[y_test.index, 'MSR_DT'])

# 중복된 인덱스 확인 및 제거
X_train = X_train[~X_train.index.duplicated(keep='first')]
X_test = X_test[~X_test.index.duplicated(keep='first')]
y_train = y_train[~y_train.index.duplicated(keep='first')]
y_test = y_test[~y_test.index.duplicated(keep='first')]

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Columns in the DataFrame: Index(['MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2', 'Year', 'Month',
       'Hour'],
      dtype='object')
Shape of X_train: (25185, 7)
Shape of X_test: (25184, 7)
Shape of y_train: (25185, 2)
Shape of y_test: (25184, 2)


/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-

                     test_PM25  pred_PM25  test_PM10  pred_PM10
MSR_DT                                                         
2023-09-09 04:00:00   0.009128   0.014986   0.006549   0.014315
2022-07-26 17:00:00   0.018256   0.012072   0.011587   0.012392
2023-05-29 15:00:00   0.007099   0.011306   0.005038   0.006293
2021-04-26 11:00:00   0.005071   0.018080   0.008564   0.013996
2022-08-04 16:00:00   0.012170   0.009824   0.008564   0.006759


ValueError: Found input variables with inconsistent numbers of samples: [25184, 25185]

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# 각 CSV 파일 읽기
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# 데이터프레임 리스트
dfs = [df_21, df_22, df_23]

# 데이터프레임 결합
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

# MSR_DT를 datetime 형식으로 변환
df['MSR_DT'] = pd.to_datetime(df['MSR_DT'], format='%Y%m%d%H', errors='coerce')

# 'Year', 'Month', 'Hour' 열 추가
df['Year'] = df['MSR_DT'].dt.year
df['Month'] = df['MSR_DT'].dt.month
df['Hour'] = df['MSR_DT'].dt.hour

# 결측값 확인 및 처리
for col in df.columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# 중복된 인덱스 제거
df = df[~df.index.duplicated(keep='first')]

# 열 이름 확인
print("Columns in the DataFrame:", df.columns)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 타입 확인 및 변환
df[numeric_columns] = df[numeric_columns].astype(float)
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Hour'] = df['Hour'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(df.loc[X_train.index, 'MSR_DT'])
X_test.index = pd.to_datetime(df.loc[X_test.index, 'MSR_DT'])
y_train.index = pd.to_datetime(df.loc[y_train.index, 'MSR_DT'])
y_test.index = pd.to_datetime(df.loc[y_test.index, 'MSR_DT'])

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Columns in the DataFrame: Index(['MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2', 'Year', 'Month',
       'Hour'],
      dtype='object')
Shape of X_train: (897875, 7)
Shape of X_test: (224469, 7)
Shape of y_train: (897875, 2)
Shape of y_test: (224469, 2)


/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-

ValueError: cannot reindex on an axis with duplicate labels

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# 각 CSV 파일 읽기
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# 데이터프레임 리스트
dfs = [df_21, df_22, df_23]

# 데이터프레임 결합
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

# MSR_DT를 datetime 형식으로 변환
df['MSR_DT'] = pd.to_datetime(df['MSR_DT'], format='%Y%m%d%H', errors='coerce')

# 'Year', 'Month' 열 추가
df['Year'] = df['MSR_DT'].dt.year
df['Month'] = df['MSR_DT'].dt.month

# 결측값 확인 및 처리
for col in df.columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# 월별 평균 계산
monthly_avg = df.groupby(['Year', 'Month']).mean().reset_index()

# 열 이름 확인
print("Columns in the DataFrame:", monthly_avg.columns)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
monthly_avg[numeric_columns] = scaler.fit_transform(monthly_avg[numeric_columns])

# 데이터 타입 확인 및 변환
monthly_avg[numeric_columns] = monthly_avg[numeric_columns].astype(float)
monthly_avg['Year'] = monthly_avg['Year'].astype(int)
monthly_avg['Month'] = monthly_avg['Month'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = monthly_avg.drop(columns=target_columns + ['Year', 'Month'])  # 예측에 사용될 피처
y = monthly_avg[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(monthly_avg.loc[X_train.index, ['Year', 'Month']].assign(Day=1))
X_test.index = pd.to_datetime(monthly_avg.loc[X_test.index, ['Year', 'Month']].assign(Day=1))
y_train.index = pd.to_datetime(monthly_avg.loc[y_train.index, ['Year', 'Month']].assign(Day=1))
y_test.index = pd.to_datetime(monthly_avg.loc[y_test.index, ['Year', 'Month']].assign(Day=1))

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Columns in the DataFrame: Index(['Year', 'Month', 'MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2'], dtype='object')
Shape of X_train: (28, 5)
Shape of X_test: (8, 5)
Shape of y_train: (28, 2)
Shape of y_test: (8, 2)


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# 각 CSV 파일 읽기
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# 데이터프레임 리스트
dfs = [df_21, df_22, df_23]

# 데이터프레임 결합
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

# MSR_DT를 datetime 형식으로 변환
df['MSR_DT'] = pd.to_datetime(df['MSR_DT'], format='%Y%m%d%H', errors='coerce')

# 'Year', 'Month' 열 추가
df['Year'] = df['MSR_DT'].dt.year
df['Month'] = df['MSR_DT'].dt.month

# 결측값 확인 및 처리
for col in df.columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# 데이터 타입 변환
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 월별 평균 계산
monthly_avg = df.groupby(['Year', 'Month']).mean().reset_index()

# 열 이름 확인
print("Columns in the DataFrame:", monthly_avg.columns)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
monthly_avg[numeric_columns] = scaler.fit_transform(monthly_avg[numeric_columns])

# 데이터 타입 확인 및 변환
monthly_avg[numeric_columns] = monthly_avg[numeric_columns].astype(float)
monthly_avg['Year'] = monthly_avg['Year'].astype(int)
monthly_avg['Month'] = monthly_avg['Month'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = monthly_avg.drop(columns=target_columns + ['Year', 'Month'])  # 예측에 사용될 피처
y = monthly_avg[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(monthly_avg.loc[X_train.index, ['Year', 'Month']].assign(Day=1))
X_test.index = pd.to_datetime(monthly_avg.loc[X_test.index, ['Year', 'Month']].assign(Day=1))
y_train.index = pd.to_datetime(monthly_avg.loc[y_train.index, ['Year', 'Month']].assign(Day=1))
y_test.index = pd.to_datetime(monthly_avg.loc[y_test.index, ['Year', 'Month']].assign(Day=1))

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Columns in the DataFrame: Index(['Year', 'Month', 'MSR_DT', 'PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2'], dtype='object')
Shape of X_train: (28, 5)
Shape of X_test: (8, 5)
Shape of y_train: (28, 2)
Shape of y_test: (8, 2)


/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-

            test_PM25  pred_PM25  test_PM10  pred_PM10
2023-12-01   0.597356        NaN   0.274462        NaN
2022-02-01   0.667229        NaN   0.274755        NaN
2023-03-01   1.000000        NaN   0.855225        NaN
2023-07-01   0.281779        NaN   0.087614        NaN
2022-05-01   0.522687        NaN   0.301264        NaN


ValueError: Found input variables with inconsistent numbers of samples: [8, 28]

In [16]:
X_test

,MSR_DT,O3,CO,SO2,NO2
2023-12-01,1.702745e+18,0.104518,0.487550,0.291272,0.746690
2022-02-01,1.644887e+18,0.443615,0.464916,0.670210,0.533617
2023-03-01,1.678968e+18,0.526599,0.428907,0.695784,0.794924
2023-07-01,1.689493e+18,0.169320,0.040026,0.192383,0.164395
2022-05-01,1.652702e+18,1.000000,1.000000,0.954882,0.280084
2023-08-01,1.692183e+18,0.223879,0.064875,0.152415,0.064467
2022-10-01,1.665925e+18,0.277261,0.094791,0.345097,0.294270
2022-01-01,1.642334e+18,0.092741,0.722527,1.000000,1.000000


In [18]:
X_train

,MSR_DT,O3,CO,SO2,NO2
2021-09-01,1.631755e+18,0.342992,0.101581,0.373912,0.026811
2022-06-01,1.655338e+18,0.507907,0.024706,0.330675,0.041344
2021-10-01,1.634386e+18,0.249009,0.187840,0.427727,0.344894
2023-11-01,1.700089e+18,0.194939,0.328496,0.332116,0.508818
2021-01-01,1.610807e+18,0.114702,0.497228,0.802599,0.785838
2021-05-01,1.621167e+18,0.673625,0.205723,0.439728,0.229331
2023-06-01,1.686876e+18,0.656714,0.094269,0.131383,0.197154
2022-04-01,1.650068e+18,0.640658,0.252566,0.602011,0.381078
2022-08-01,1.660654e+18,0.234508,0.060978,0.338378,0.063922
2021-06-01,1.623809e+18,0.651689,0.165285,0.614552,0.235307
